# Constructing a Dobble card set

The set of cards of a Dobble game is defined by the rules

1. Any two cards have exactly one symbol in common.
2. Any two symbols appear together on exactly one card.

It turns out that these requirements have an exact geometrical analogue. In two-dimensional affine geometry, the following two rules hold:

1. Any two non-parallel lines meet in exactly one point.
2. Any two points lie on (define) exactly one line. 

Since these characteristics depend only on the underlying vector-space, they particularly hold for vector-spaces over *finite fields* $\mathbb{F}_q$ with $q$ elements. So the strategy to construct a Dobble card game is as follows. Associate "card" with "line" and "symbol" with "point" and consider the affine plane $\mathbb{A}^2_q$ over the vector space $\mathbb{F}_q^2$. Next, construct every line in $\mathbb{A}^2_q$. Consider therefore that every line is defined by an equation
$$
    \mathbf{w}\cdot\mathbf{x} + b = 0 \qquad\qquad (1)
$$
for some $\mathbf{w} \in \mathbb{F}_q^2$ and $b\in \mathbb{F}_q$. In other words, each line is defined by a tuple $(\mathbf{w}, b)$ where $\mathbf{w}$ ranges over all different "directions" of the vector space. It is easily verified that 
$$
    W := \left\{ (\bar 0,\bar 1)^T \right\} \cup \left\{ (\bar 1, \bar 0)^T, (\bar 1, \bar 2)^T, \ldots, (\bar 1,\overline{q-1})^T \right\} \subset \mathbb{F}_q^2
$$
is a complete and linear independent set representing all the directions in $\mathbb{A}_q ^2$. 
With basic linear algebra one can show that the $p^2 + p$ tuples $(\mathbf{w}, b) \in W \times \mathbb{F}_q$ indeed define all the lines in 
$\mathbb{A}_q^2$. Note that for every vector $\mathbf{x} = (x_1,x_2)^T$ in equation (1) it is true that if you fix any component $x_1$ or $x_2$, there is exactly one solution for the other one. It follows that every line contains exactly $q$ points. 

Now, two different lines intersect in one point if they are _not parallel_, i.e. they have different norm vectors $\mathbf{w}$, 
whereas two lines with the same directions don't have a point in common in the affine plane. 
To make sure that also in the parallel case two lines share an equal point, we assign a new element $p_{\mathbf{w}} 
\not\in\mathbb{A}_q^2$, called a "point at infinity", to every line with direction $\mathbf{w}$. With this postulation, any two lines have exactly one point in common, whether they share the same direction or not. Finally, we add another line containing exactly all the "points at infinity". This makes sure that any two points lie on exactly one line.  (Technically, we are moving on to the _projective plane_ over $\mathbb{F}_q$, but for our purpose we can treat these infinity points and lines as simple _ad hoc_ postulations because at this point of the argument we don't need any algebraic structure of the point set anymore).

This finishes the construction of the Dobble card game. Substituting "card" with "line" and "point" with symbol, we see that required properties for the card set hold. Numberwise, the set has the following properties:

* Every card contains $q+1$ symbols. (Every line contains $q$ normal points plus one "point at infinity".)
* There are $q^2 + q + 1$ different symbols. (The $q^2$ points in $\mathbb{A}_q^2$ plus $q+1$ "points at infinity".)
* There are $q^2 + q + 1$ different cards. (The $q^2 + q$ lines in $\mathbb{A}_q^2$ plus one "line at infinity".)

Note that the construction depends on the existence of a field $\mathbb{F}_q$ with $q$ elements. Such a field exists if and only if $q$ is a prime power, i.e. $q=p^n$ for some prime number $n$. For $n=1$ this is simply the residual ring $\mathbb{Z}/p\mathbb{Z}$ where the arithmetic is defined by modulo $p$. But in general, for $n>1$, the field $\mathbb{F}_q$ is not isomorphic to the the residual ring. 

In [130]:
import random
import galois
import numpy as np

class Line:
    def __init__(self, GF, w, b):
        self.w = GF(w)
        self.b = GF(b)
        # The points on the line. 
        # All parallel lines (lines with equal w) have a "point at infinity" in common which, 
        # by convention, we denote with (-w_0, -w_1)
        self.points = [np.array([-w[0],-w[1]])]
        
    def contains(self, x):
        return np.dot(self.w, x) + self.b == 0

    def print(self):
        print(f"w={self.w.tolist()}x+{self.b.tolist()}: Points: {[x.tolist() for x in self.points]}")

class Dobble:

    # q is the order of the finite field. Therefore, q has to be a prime power, q=p^n for some prime p.
    # Every card will contain q+1 symbols
    def __init__(self, q):
        self.GF = galois.GF(q)
        # In a finite affine vector space over a field of order n there are:
        #   -- n+1 different directions: w=(0,1),(1,0),...(1,n-1)
        #   -- n different offsets: b=0,...,n-1
        # This makes n^2+n different lines defined by wx+b=0
        self.lines = [Line(self.GF,(0,1),b) for b in range(q)] + [Line(self.GF,(1,m),b) for m in range(q) for b in range(q)]
        self.points = [self.GF((x,y)) for x in range(q) for y in range(q)]

        # Assign the points to the line
        # TODO: optimize! This is highly redundant. Simply compute $w*x$ for every direction and assign to the appropriate line.  
        for line in self.lines:
            for x in self.points:
                if line.contains(x):
                    line.points.append(x)

        # Add the "line at infinity" containing all "points at infinity"
        line_at_infinity = Line(self.GF,(0,0),0)
        line_at_infinity.points = [np.array([0,-1])] + [np.array([-1,-m]) for m in range(q)]
        self.lines.append(line_at_infinity)

    def play(self):
        i,j = random.sample(range(len(self.lines)), 2)
        common_points = []
        for x in self.lines[i].points:
            for y in self.lines[j].points:
                if x[0]==y[0] and x[1]==y[1]:
                    common_points.append(x)
        
        print("New Game")
        self.lines[i].print()
        self.lines[j].print()
        print(F"Common point {[x.tolist() for x in common_points]}")
        print("")

In [132]:
dobble = Dobble(7)

for i in range(5):
    dobble.play()

[GF([0, 0], order=7), GF([0, 1], order=7), GF([0, 2], order=7), GF([0, 3], order=7), GF([0, 4], order=7), GF([0, 5], order=7), GF([0, 6], order=7), GF([1, 0], order=7), GF([1, 1], order=7), GF([1, 2], order=7), GF([1, 3], order=7), GF([1, 4], order=7), GF([1, 5], order=7), GF([1, 6], order=7), GF([2, 0], order=7), GF([2, 1], order=7), GF([2, 2], order=7), GF([2, 3], order=7), GF([2, 4], order=7), GF([2, 5], order=7), GF([2, 6], order=7), GF([3, 0], order=7), GF([3, 1], order=7), GF([3, 2], order=7), GF([3, 3], order=7), GF([3, 4], order=7), GF([3, 5], order=7), GF([3, 6], order=7), GF([4, 0], order=7), GF([4, 1], order=7), GF([4, 2], order=7), GF([4, 3], order=7), GF([4, 4], order=7), GF([4, 5], order=7), GF([4, 6], order=7), GF([5, 0], order=7), GF([5, 1], order=7), GF([5, 2], order=7), GF([5, 3], order=7), GF([5, 4], order=7), GF([5, 5], order=7), GF([5, 6], order=7), GF([6, 0], order=7), GF([6, 1], order=7), GF([6, 2], order=7), GF([6, 3], order=7), GF([6, 4], order=7), GF([6, 5], o

In [118]:
w = [(0,1)] + [(x,y) for x in range(3) for y in range(3)]
p = [[ww, [[] for _ in range(3)]] for ww in w]
print(p)

[[(0, 1), [[], [], []]], [(0, 0), [[], [], []]], [(0, 1), [[], [], []]], [(0, 2), [[], [], []]], [(1, 0), [[], [], []]], [(1, 1), [[], [], []]], [(1, 2), [[], [], []]], [(2, 0), [[], [], []]], [(2, 1), [[], [], []]], [(2, 2), [[], [], []]]]
